In [0]:
box::use(dplyr[...], sparklyr[...])
sc <- spark_connect(method = "databricks")
options('box.path'='/Workspace/Users/joe.wang@gatesfoundation.org/DHS-Indicators-R')

In [0]:
box::use(Chap07_FP/chapter)

In [0]:
data <- chapter$load_data(sc, NULL, c("ET"))

In [0]:
indicators <- chapter$run(data)

In [0]:
merged <- chapter$process(indicators)

In [0]:
box::reload(chapter)

In [0]:
write_indicators <- function(sc, chapter, indicators) {
  tbl_indicators <- copy_to(sc, indicators, "_indicators", overwrite = TRUE)
  countries <- unique(indicators$`_cc`)

  spark_write_table(
    tbl_indicators,
    name = paste0("dhs.indicator.chapter_", chapter$get_config()$chapter),
    mode = "overwrite",
    options = list(
      mergeSchema = "true",
      replaceWhere = paste0("_cc IN ('", paste0(countries, collapse="','", "')"))
    )
  )
}

In [0]:
write_indicators(sc, chapter, merged)